# GEOG 313 Final




Function 1: Read in Sentinel-1 SAR 

	- Inputs: AOI, date range 
	- Outputs: Median quarterly composites (mosaiced) of VH and VV bands 

Function 2: Create index time series 

	- Inputs: median quarterly composites, name of index (RVI, mRFDI) 
	- Outputs: time series of index value 

Function 3: Create Visualizations within subzones 

	- Inputs: time series, subzones 
	- outputs: graph of index value vs time for each subzone. 

Function 4: Visualize trends 

	- Inputs: time series, name of trend (linear, monotonic, etc) 
	- outputs: raster image of correlation coefficient or other values (depending on what trend is used- ex. for linear trend it would output two images, r value and correlation coefficient) 

Function 5: detect deforestation 

	- Inputs: time series, index threshold value 
	- outputs: image of what year deforestation occurred (when value decreased below the user-defined threshold. If the value decreased then increased the result will be inconclusive and marked with an appropriate value) 



In [2]:
import dask
from dask.distributed import Client, LocalCluster
cluster = LocalCluster()
client = Client(cluster)
print(client.dashboard_link)

http://127.0.0.1:8787/status


In [6]:
aoi = [38.5, -5.25, 38.75,-4.75]

start = "2020-01-01"
end = "2020-12-31"

bands = ["VV", "VH"]

In [11]:
search = get_sar_pc(aoi, start, end)
search

subset = search[0]
subset

<Item id=S1B_IW_GRDH_1SDV_20201227T030420_20201227T030445_024882_02F5EA_rtc>

In [13]:

stack = stackstac.stack(
    subset,
    epsg=32737,
    resolution=20,
    )

TypeError: can_cast() does not support Python ints, floats, and complex because the result used to depend on the value.
This change was part of adopting NEP 50, we may explicitly allow them again in the future.

In [4]:
import xarray as xr
import rioxarray as rxr
import numpy as np
import stackstac
import os
import leafmap
import requests
from tqdm import tqdm
import pystac_client
import planetary_computer
from matplotlib import pyplot as plt

# write function to get bbox for polygons


def get_sar_pc(bbox, start, end):

    catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",  # connect to Planetary Computer API
    modifier=planetary_computer.sign_inplace,               
    )

    search = catalog.search(
        collections=["sentinel-1-rtc"],       # get sentienel 2 l2a
        bbox=bbox,                             # within bbox
        datetime = f"{start}/{end}",          # within specified dates
       #query = ["sar:polarizations[0]"],
    )

    items = search.get_all_items()  # get items into new variable
    return items


def items_to_xarray(items, bands):
    stack = stackstac.stack(
        items,                 # output of item collection search
   #     bounds_latlon=bbox,       # clip to bbox
        assets=bands           # get only green and nir bands
    )
    
    return stack

def quartely_composite(stack):
    composite = stack.resample(time="1Q").median
